In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scipy.io import arff

data, _ = arff.loadarff('dataset_50_tic-tac-toe.arff')

In [ ]:
df = pd.DataFrame(data)
df.head()

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,Class
0,b'x',b'x',b'x',b'x',b'o',b'o',b'x',b'o',b'o',b'positive'
1,b'x',b'x',b'x',b'x',b'o',b'o',b'o',b'x',b'o',b'positive'
2,b'x',b'x',b'x',b'x',b'o',b'o',b'o',b'o',b'x',b'positive'
3,b'x',b'x',b'x',b'x',b'o',b'o',b'o',b'b',b'b',b'positive'
4,b'x',b'x',b'x',b'x',b'o',b'o',b'b',b'o',b'b',b'positive'


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   top-left-square       958 non-null    object
 1   top-middle-square     958 non-null    object
 2   top-right-square      958 non-null    object
 3   middle-left-square    958 non-null    object
 4   middle-middle-square  958 non-null    object
 5   middle-right-square   958 non-null    object
 6   bottom-left-square    958 non-null    object
 7   bottom-middle-square  958 non-null    object
 8   bottom-right-square   958 non-null    object
 9   Class                 958 non-null    object
dtypes: object(10)
memory usage: 75.0+ KB


In [ ]:
df = df.apply(lambda x: x.str.decode('utf-8'))
df.head()

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [ ]:
df.isnull().sum()

top-left-square         0
top-middle-square       0
top-right-square        0
middle-left-square      0
middle-middle-square    0
middle-right-square     0
bottom-left-square      0
bottom-middle-square    0
bottom-right-square     0
Class                   0
dtype: int64

In [ ]:
X = df.drop(columns='Class')
y = df['Class']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
X_transformed = ohe.fit_transform(X).toarray()

In [ ]:
y = y.apply(lambda x: 1 if x == 'positive' else 0)

0      1
1      1
2      1
3      1
4      1
      ..
953    0
954    0
955    0
956    0
957    0
Name: Class, Length: 958, dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

ohe = OneHotEncoder()
X_transformed = ohe.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=.2) 

lr = LogisticRegression().fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

0.9635416666666666

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2) 

rfc = RandomForestClassifier().fit(X_train, y_train)
rfc.score(X_test, y_test)

0.96875

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder()
X_transformed = ore.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=.2) 

rfc = RandomForestClassifier().fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9583333333333334

In [ ]:
from sklearn.tree import DecisionTreeClassifier


ohe = OneHotEncoder()
X_transformed = ohe.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=.2) 

dtc = DecisionTreeClassifier().fit(X_train, y_train)
dtc.score(X_test, y_test)

0.9322916666666666

In [ ]:
ore = OrdinalEncoder()
X_transformed = ore.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=.2) 

dtc = DecisionTreeClassifier().fit(X_train, y_train)
dtc.score(X_test, y_test)

0.90625

In [ ]:
def train_and_get_score(model, encoder=None, X=X, y=y, test_size=.2):
    if encoder == 'ohe':
        X_transformed = OneHotEncoder().fit_transform(X)
    elif encoder == 'ore':
        X_transformed = OrdinalEncoder().fit_transform(X)
    else:
        X_transformed = X
        
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=test_size) 
        
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid_rfc = {
    'max_depth': [5, 10, 15],
    'n_estimators': [100, 200, 300],
    'min_samples_split': [15, 20],
    'min_samples_leaf': [1, 10, 12]
}

rfc = RandomForestClassifier(random_state=23)
rfc_cv = GridSearchCV(rfc, param_grid_rfc, cv=5, n_jobs=-1)

train_and_get_score(rfc_cv)

0.9427083333333334
